# Baseline: Обучение Multi-Branch MLP на размеченных данных


In [1]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight

from model import MultiBranchMLP
from data_module import DataModule
from lightning_module import BaseLightningModule

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    import random
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


## 1. Загрузка данных


In [2]:
data_dir = '../data'

dm = DataModule(
    data_dir=data_dir,
    batch_size=128,
    num_workers=4
)

dm.setup()

print(f'Input dimension: {dm.input_dim}')
print(f'Number of classes: {dm.n_classes}')
print(f'Labeled train samples: {len(dm.train_labeled_dataset)}')
print(f'Test samples: {len(dm.test_dataset)}')


Input dimension: 3072
Number of classes: 10
Labeled train samples: 1600
Test samples: 4000
Input dimension: 3072
Number of classes: 10
Labeled train samples: 1600
Test samples: 4000


## 2. Анализ дисбаланса классов и вычисление весов


In [3]:
train_labels = dm.train_labeled_dataset.y

unique_labels = np.unique(train_labels)
class_weights = compute_class_weight(
    'balanced',
    classes=unique_labels,
    y=train_labels
)

print(f'Class weights: {dict(zip(unique_labels, class_weights))}')

class_weights_tensor = torch.FloatTensor(class_weights)


Class weights: {np.int64(0): np.float64(1.0738255033557047), np.int64(1): np.float64(0.963855421686747), np.int64(2): np.float64(1.0256410256410255), np.int64(3): np.float64(1.103448275862069), np.int64(4): np.float64(0.9523809523809523), np.int64(5): np.float64(0.935672514619883), np.int64(6): np.float64(0.9523809523809523), np.int64(7): np.float64(1.0596026490066226), np.int64(8): np.float64(0.9248554913294798), np.int64(9): np.float64(1.0457516339869282)}


## 3. Создание модели


In [4]:
model = MultiBranchMLP(
    input_dim=dm.input_dim,
    hidden_dim=256,
    output_dim=dm.n_classes,
    num_blocks=4,
    dropout=0.1,
    combine_mode='concat'
)

print(f'Model parameters: {sum(p.numel() for p in model.parameters()):,}')


Model parameters: 4,080,650


## 4. Создание Lightning модуля


In [5]:
loss_fn = nn.CrossEntropyLoss(weight=class_weights_tensor)

lightning_model = BaseLightningModule(
    model=model,
    loss_fn=loss_fn,
    optimizer_type='adamw',
    learning_rate=1e-3,
    task_type='multiclass'
)


## 5. Обучение модели


In [6]:
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    filename='best_model-{epoch:02d}-{val_accuracy:.4f}',
    monitor='val_accuracy',
    mode='max',
    save_top_k=1,
    save_last=True
)

trainer = Trainer(
    max_epochs=100,
    callbacks=[checkpoint_callback],
    enable_checkpointing=True,
    logger=True,
    enable_progress_bar=True,
    enable_model_summary=True,
    accelerator='auto',
    devices='auto'
)

trainer.fit(lightning_model, dm)


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Input dimension: 3072
Number of classes: 10
Labeled train samples: 1600
Test samples: 4000



  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | MultiBranchMLP   | 4.1 M  | train
1 | loss_fn | CrossEntropyLoss | 0      | train
2 | metrics | ModuleDict       | 0      | train
-----------------------------------------------------
4.1 M     Trainable params
0         Non-trainable params
4.1 M     Total params
16.323    Total estimated model params size (MB)
70        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Epoch 0: accuracy=0.1250, f1_macro=0.0485


...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 98: accuracy=0.2604, f1_macro=0.2618


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 99: accuracy=0.2608, f1_macro=0.2621


`Trainer.fit` stopped: `max_epochs=100` reached.


## 6. Оценка на тестовой выборке


In [7]:
best_model_path = checkpoint_callback.best_model_path
print(f'Loading best model from: {best_model_path}')

if best_model_path:
    best_model = BaseLightningModule.load_from_checkpoint(
        best_model_path,
        model=model,
        loss_fn=loss_fn,
        optimizer_type='adamw',
        learning_rate=1e-3,
        task_type='multiclass'
    )
else:
    best_model = lightning_model

test_results = trainer.test(best_model, dm)

print('\n=== Финальные результаты на тестовой выборке ===')
for key, value in test_results[0].items():
    print(f'{key}: {value:.4f}')


Loading best model from: /home/tam2511/mounts/0/arcadia/market/robotics/cv/ml/user_data/shad/dl2025/lesson7/homework/baseline/checkpoints/best_model-epoch=65-val_accuracy=0.2787.ckpt


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Input dimension: 3072
Number of classes: 10
Labeled train samples: 1600
Test samples: 4000


Testing: |          | 0/? [00:00<?, ?it/s]

Test results: accuracy=0.2830, f1_macro=0.2737


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.28299999237060547    │
│       test_f1_macro       │    0.27369052171707153    │
│         test_loss         │    13.638334274291992     │
└───────────────────────────┴───────────────────────────┘


=== Финальные результаты на тестовой выборке ===
test_loss: 13.6383
test_accuracy: 0.2830
test_f1_macro: 0.2737
